In [5]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
import stumpy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("DATASET.CSV")
window_size = 40  # Approximately, how many data points might be found in a pattern

matrix_profile = stumpy.stump(df.Close, m=window_size)
# array of [[min_distance_score, index, left, right]]

minscore = matrix_profile[:, 0].min()
# min = matrix_profile[:, 0].argmin()
topn = np.argsort(matrix_profile[:, 0])[:10]
print ((topn, minscore))

df["Mp"] = np.concatenate([matrix_profile[:, 0], np.array([0]*(window_size-1))])
# df[["Close", "Mp"]].plot(logy=True) # , sharex=True)
# plt.show()

(array([14123,  7556,  9970, 13247,  7555, 14122,  9971, 13248,  9969,
       13246], dtype=int64), 0.6822388986239004)


In [6]:
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

selected_chain = []
# fig = px.line(x=df.index, y=df.Close)
fig = px.line(df, x=df.index, y=df.Close)
# scatter = fig.add_scatter(x=df.index, y=df.Mp, hovertext=df.Date + " " + df.DayName, mode="lines", name="Matrix Profile")
scatter = fig.add_scatter(x=df.index, y=df.Mp, hovertext=df.Date, mode="lines", name="Matrix Profile")
fig.add_scatter(x=topn, y=[df.iloc[x]["Mp"] for x in topn], mode="markers", name="Minima")
fig.update_layout(yaxis_type="log")

def highlight_zones(chain):
    print (chain)
    shapes = []
    for index, hn in enumerate(chain):
        if index == 0:
            fillcolor = "LightSalmon"
        else:
            fillcolor = "Violet"
        highlight = dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=hn,
                y0=0,
                x1=hn+window_size,
                y1=1,
                fillcolor=fillcolor,
                opacity=0.5,
                layer="below",
                line_width=0,
            )
        shapes.append(highlight)
    fig.update_layout(shapes=shapes)
    selected_chain = chain
    return fig
    
# https://plotly.com/python/shapes/
def fn(h1=topn[0]):
    h2 = matrix_profile[h1, 1]
    return highlight_zones([h1, h2])

from IPython.display import display
# h1_input = (0, len(df))
h1_input_txt = widgets.IntText(value=topn[0])
h1_input_slider = widgets.IntSlider(continuous_update=False, min=0, max=len(df), value=topn[0])
display(h1_input_slider)
h1_link = widgets.jslink((h1_input_txt, 'value'), (h1_input_slider, 'value'))
# interact_manual(fn, h1=h1_input)

interact(fn, h1=h1_input_txt)


IntSlider(value=14123, continuous_update=False, max=19335)

interactive(children=(IntText(value=14123, description='h1'), Output()), _dom_classes=('widget-interact',))

<function __main__.fn(h1=14123)>

In [32]:
mp = matrix_profile
anchored_chain = stumpy.atsc(mp[:, 2], mp[:, 3], h1_input_slider.value)

print ( (anchored_chain, h1_input_slider.value) )

fig = highlight_zones(anchored_chain)
selected_chain = anchored_chain
fig

(array([308, 867], dtype=int64), 308)
[308 867]


In [7]:
mp = matrix_profile
all_chain_set, unanchored_chain = stumpy.allc(mp[:, 2], mp[:, 3])
print (unanchored_chain)
print (len(df))
# print (all_chain_set)
selected_chain = unanchored_chain

fig = highlight_zones(unanchored_chain)
fig

[  622  3043  5618 14131 15900 17817]
19335
[  622  3043  5618 14131 15900 17817]


In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

NUM_WINDOWS = 2
h1 = h1_input_txt.value
h2 = matrix_profile[h1, 1]

if len(selected_chain) > 2:
    chain = selected_chain
else:
    chain = [h1, h2]
fig = make_subplots(rows=len(chain), cols=1, shared_xaxes=True)
for index, hn in enumerate(chain):
    df_hn = df[hn:hn+window_size*NUM_WINDOWS].reset_index()
    fig.add_scatter(
        x=df_hn.index, y=df_hn.Close,
        row=index+1, col=1,
        name="slice" + str(index)
    )

fig.update_layout(height=600, width=800, title_text="Subplots")
# fig.update_xaxes(title_text="xaxis 2 title", range=[10, 50], row=1, col=2)
fig.show()